# PET data case

Tested with CIL 21.1.0 and SIRF 3.0.0

The data is a real dataset of a phantom acquisition at UCL on a Siemens mMR. The phantom is the NEMA phantom (essentially a torso-shaped perspex box, with some spherical inserts).  The data is freely available on [Zenodo](https://zenodo.org/record/1304454#.YJU7xbVKg10).

The data must be downloaded and put in a the directory pointed by `data_path`.

Parts of this notebook are taken from the SIRF exercise [reconstruct_measured_data ](https://github.com/SyneRBI/SIRF-Exercises/blob/master/notebooks/PET/reconstruct_measured_data.ipynb) from CCP SyneRBI.

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import sirf.STIR as pet
from sirf.config import SIRF_HAS_Parallelproj

from cil.utilities.display import show2D
from cil.optimisation.algorithms import PDHG, SPDHG
from cil.optimisation.functions import KullbackLeibler
from cil.plugins.ccpi_regularisation.functions import FGP_TV
from ccpi.filters import regularisers

# change data_path to point to the directory where the data has been downloaded and unzipped
data_path = os.path.abspath('/home/ofn77899/devel/buildVM/INSTALL/share/SIRF-3.0/data/examples/PET/mMR_newserver')
os.chdir(data_path)

In [ ]:
# file names
list_file = '20170809_NEMA_60min_UCL.l.hdr'
norm_file = 'norm.n.hdr'
attn_file = 'mu_map.hv'
# output filename prefixes
sino_file = 'sino'

In [ ]:
# redirect STIR messages to some files
# you can check these if things go wrong
msg_red = pet.MessageRedirector('info.txt', 'warn.txt')

In [ ]:
template_acq_data = pet.AcquisitionData('Siemens_mMR', span=11, max_ring_diff=15, view_mash_factor=1)
template_acq_data.write('template.hs')
# create listmode-to-sinograms converter object
lm2sino = pet.ListmodeToSinograms()

# set input, output and template files
lm2sino.set_input(list_file)
lm2sino.set_output_prefix(sino_file)
lm2sino.set_template('template.hs')
# set timing interval (in secs) since start of acquisition
# (the listmode file provided is for 1 hour).
# you can vary this to see the effect on noise. Increasing it will mean somewhat longer
# processing time in the following steps (but not in the reconstruction).
lm2sino.set_time_interval(0, 500)
# set up the converter
lm2sino.set_up()
# create the prompts sinogram
lm2sino.process()
# get access to the sinograms
# acq_data = lm2sino.get_output()
acq_data = pet.AcquisitionData('sino_f1g1d0b0.hs')

# use a slice number for display that is appropriate for the NEMA phantom
z = 71

# uncomment to show the acquisition data
# show2D(acq_data, slice_list=[(0,0),(1,z)], title='500s', cmap='gray_r' )

In [ ]:
# Get the randoms estimate
# This will take a while
rand = lm2sino.estimate_randoms()
rand.write('sino_randoms_f1g1d0b0')

# uncomment to show the randoms
# show2D(rand, slice_list=[(0,0),(1,z)], title='randoms 500s', cmap='gray_r')

## OSEM reconstruction

In [ ]:
# Acquisition data stored in memory
pet.AcquisitionData.set_storage_scheme('memory')
pet.set_verbosity(0)

# attenuation
attns = pet.ImageData('mu_map.hv')
asm_norm = pet.AcquisitionSensitivityModel('norm.n.hdr')

# Setup image geometry
image = acq_data.create_uniform_image(0., (127, 220, 220))
image.initialise(dim=(127, 220, 220), vsize=(2.03125, 1.7080754, 1.7080754))

def get_asm_attn(sino, attn, acq_model):
    """Get attn Attenuation Sensitivity Model from sino, attn image and acq model."""
    asm_attn = pet.AcquisitionSensitivityModel(attn, acq_model)
    # temporary fix pending attenuation offset fix in STIR:
    # converting attenuation into 'bin efficiency'
    asm_attn.set_up(sino)
    bin_eff = pet.AcquisitionData(sino)
    bin_eff.fill(1.0)
    asm_attn.unnormalise(bin_eff)
    asm_attn = pet.AcquisitionSensitivityModel(bin_eff)
    return asm_attn

# set up the acquisition model
if SIRF_HAS_Parallelproj:
    # use a GPU implementation if available
    am = pet.AcquisitionModelUsingParallelproj()
else:
    am = pet.AcquisitionModelUsingRayTracingMatrix()

# ASM norm already there
asm_attn = get_asm_attn(acq_data,attns,am)

# Get ASM dependent on attn and/or norm
asm = pet.AcquisitionSensitivityModel(asm_norm, asm_attn)
am.set_acquisition_sensitivity(asm)
am.set_background_term(rand)
am.set_up(acq_data, image)

In [ ]:
# setup OSEM
obj_fun = pet.make_Poisson_loglikelihood(acq_data)
obj_fun.set_acquisition_model(am)

recon = pet.OSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)
recon.set_num_subsets(7)
recon.set_num_subiterations(58)
recon.set_up(image)

# set the initial image estimate
recon.set_current_estimate(image)

In [ ]:
# run OSEM
recon.process()
osem_recon = recon.get_output()

In [ ]:
# show OSEM recon
# show2D(osem_recon.as_array()[72,:,:], cmap="inferno", fix_range=(0,0.15), origin="upper")

## Reconstruction with TV regularisation using PDHG and preconditioning

In [ ]:


# Get linear acquisition model
K = am.get_linear_acquisition_model()
# K.img_templ = am.img_templ
# K.acq_templ = am.acq_templ

To accelerate the reconstruction, we use diagonal preconditioning to compute $\sigma$ and $\tau$ based on the operator K. See [here](https://ieeexplore.ieee.org/document/6126441) for more information.

In [ ]:
tmp_sigma = K.direct(K.domain_geometry().allocate(1.)).power(-1)
tmp_tau   = K.adjoint(K.range_geometry().allocate(1.)).power(-1)

tmp_tau_np = tmp_tau.as_array()
tmp_tau_np[tmp_tau_np==np.inf] = 1e-5
tau = tmp_tau*0.
tau.fill(tmp_tau_np)

tmp_sigma_np = tmp_sigma.as_array()
tmp_sigma_np[tmp_sigma_np==np.inf] = 1e-5
sigma = tmp_sigma*0.
sigma.fill(tmp_sigma_np)

The implementation of `FGP_TV` does not accept $\tau$ as a `numpy` array. Hence, we change its `proximal` method to a `precond_proximal` method. 

In [ ]:
def precond_proximal(self,x,tau, out=None):
        in_arr = np.asarray(x.as_array()/tau.as_array(), dtype=np.float32, order='C')      
        res , info = regularisers.FGP_TV(\
              in_arr,\
              self.alpha,\
              self.max_iteration,\
              self.tolerance,\
              self.methodTV,\
              self.nonnegativity,\
              self.device)
        if out is not None:
            out.fill(res)
        else:
            out = x.copy()
            out.fill(res)
        out *= tau    
        return out    
    
FGP_TV.proximal = precond_proximal

Next, we define the fidelity term **KullbackLeibler** based on the acqusition data and random events. For the regularisation term we use the **FGP_TV** function class from the [CCPi-RegTk](https://github.com/vais-ral/CCPi-Regularisation-Toolkit).

In [ ]:
alpha = 13.0
F = KullbackLeibler(b = acq_data, eta = rand)
G = alpha * FGP_TV(max_iteration=100, device='gpu' )
max_iterations = 500 # Use 500 to match the results in the paper. 

# Setup and run PDHG
pdhg = PDHG(f = F, g = G, operator=K, sigma = sigma, tau = tau,
            max_iteration = max_iterations, update_objective_interval = 100, use_axpby=False)

In [ ]:
pdhg.run(verbose = 2, print_interval=50)

In [ ]:
plt.figure(figsize=(10,10))
ss = 72
plt.imshow(osem_recon.as_array()[ss,:,:], cmap="inferno", vmax = 0.15)
plt.title("OSEM")
plt.plot((125,125),(0,219),'-r')
plt.colorbar()
# plt.savefig("osem_annot.png",dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(pdhg.solution.as_array()[ss,:,:], cmap="inferno", vmax = 0.15)
plt.title("PDHG + TV")
plt.plot((125,125),(0,219),'-r')
plt.colorbar()
# plt.savefig("pdhgtv_annot.png",dpi=300)
plt.show()

In [ ]:
plt.figure()
plt.plot(osem_recon.as_array()[72,:,125],label='OSEM')
plt.plot(pdhg.solution.as_array()[72,:,125],label='PDHG TV')
plt.legend()
# plt.savefig('vertical_profile.png',dpi=300)
plt.show()